In [1]:
from torch.utils.data.sampler import SubsetRandomSampler
from torch.optim.lr_scheduler import ReduceLROnPlateau
from tensorboard_logger import configure, log_value
from model import RecurrentAttention
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
import matplotlib.pyplot as plt
from utils import AverageMeter
from config import get_config
from utils import plot_images
from trainer import Trainer
from tqdm import tqdm
import pandas as pd
import torchvision
import numpy as np
import random
import pickle
import shutil
import torch
import time

import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

from utils import *

In [2]:
q = 4
with open("../../pooling/data/migration_data.json") as m:
    mig_data = json.load(m)
m.close()
mig_data = pd.DataFrame.from_dict(mig_data, orient = 'index').reset_index()
mig_data.columns = ['muni_id', 'num_migrants']
mig_data['class'] = pd.qcut(mig_data['num_migrants'], q = q, labels = [i for i in range(q)])
mig_data.head()

,muni_id,num_migrants,class
0,484001001,42055.0,3
1,484001002,4017.0,3
2,484001003,11992.0,3
3,484001004,762.0,2
4,484001005,7551.0,3


In [3]:
image_names = get_png_names("../../attn/data/MEX/")
y_class, y_mig = get_ys(image_names, mig_data)

with open("../val_images.txt") as ims:
    val_names = ims.read().splitlines()
    
train_names = [i for i in image_names if i not in val_names]

val_indices = [image_names.index(i) for i in val_names]
train_indices = [image_names.index(i) for i in train_names]
train_num = int(len(train_indices))

In [4]:
# train_indices = train_indices[0:25]
# val_indices = val_indices[0:10]

In [5]:
batch_size = 1
train = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_class[i], y_mig[i]) for i in train_indices]
val = [(torchvision.transforms.functional.adjust_brightness(load_inputs(image_names[i]), brightness_factor = 2).squeeze(), y_class[i], y_mig[i]) for i in val_indices]
train_dl = torch.utils.data.DataLoader(train, batch_size = batch_size, shuffle = True)
val_dl = torch.utils.data.DataLoader(val, batch_size = batch_size, shuffle = True)

In [6]:
print("Num training: ", len(train_dl))
print("Num validation: ", len(val_dl))

Num training:  1446
Num validation:  620


In [ ]:
config, unparsed = get_config()
trainer = Trainer(config, (train_dl, val_dl))
trainer.train()

0.1s - loss: 2675.982 - acc: 0.000 - mae: 2675.983:   0%|          | 0/1446 [00:00<?, ?it/s]


[*] Train on 1446 samples, validate on 620 samples

Epoch: 1/200 - LR: 0.000300


61.9s - loss: 143.793 - acc: 0.000 - mae: 143.791: 100%|██████████| 1446/1446 [01:01<00:00, 23.35it/s]    


train loss: 1874.831 - train acc: 25.934 - val loss: 1575.046 - val acc: 24.355 - val err: 75.645- train mae: 1874.510 - val mae: 1574.922 [*]


0.2s - loss: 158.088 - acc: 0.000 - mae: 158.087:   0%|          | 2/1446 [00:00<01:56, 12.34it/s]    

...

0.2s - loss: 89.947 - acc: 0.000 - mae: 89.946:   0%|          | 3/1446 [00:00<01:12, 19.87it/s]  


Epoch: 88/200 - LR: 0.000003


60.7s - loss: 888.161 - acc: 100.000 - mae: 888.161: 100%|██████████| 1446/1446 [01:00<00:00, 23.81it/s]  
0.0s - loss: 353.343 - acc: 0.000 - mae: 353.343:   0%|          | 0/1446 [00:00<?, ?it/s]

train loss: 1558.454 - train acc: 23.651 - val loss: 1476.786 - val acc: 23.387 - val err: 76.613- train mae: 1558.453 - val mae: 1476.786

Epoch: 89/200 - LR: 0.000000


61.8s - loss: 6606.711 - acc: 0.000 - mae: 6606.710: 100%|██████████| 1446/1446 [01:01<00:00, 23.38it/s]  
